# Transaction Value Prediction
In this notebook, we will tackle the challenge presented by Santander Group in terms of predicting "the value of transactions for each potential customer". In other words, the rgoup would provide an an anonymized dataset (for security purpose) to attain this objective. In essence, this presents a regression problem which we shall resolve using different machine learning techniques.

# Table of Contents
1. Environment set-up
    * Importing Libraries
    * Loading the data
2. Initial Diagnostics
    * Glimpse
    * Descriptive Statitics
    * Target Variable Analysis
3. Data Cleaning
    * Missing Values
    * Outliers
    * Duplicate Observations
4. Correlation Analysis
5. Machine Learning set-up
    * Feature Scaling
    * Train-test split
6. Machine Learning - Ensemble Methods
    * Random Forest Regressor
    * XGBoostRegressor
7. Machine Learning - Neural Networks
8. Final Submission

# 1. Environment set-up

In [ ]:
## Importing Libraries

#Set seed
import random
random.seed(69)

# Manipulating & Visualizing Data
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(12, 8)})

# Machine Learning set-up
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold

# Cross Validation
from sklearn.model_selection import  KFold

# Ensemble Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

# Neural Networks
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

# Performance metrics
import sklearn.metrics as skm

# System file
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Loading the dataset
df = pd.read_csv("/kaggle/input/santander-value-prediction-challenge/train.csv")

#Removing the unnecessary ID column
df.drop("ID", axis=1, inplace=True)

# 2. Initial Diagnostics

In [ ]:
## Glimpse of the data
df.head()

In [ ]:
## Structure of dataframe
df.info()

**Note:** The histogram below shows a right-skewed distribution indicating that most transactions are on the lower end. It remains consistent given that banking transaction for x person (on average) is relatively modest, with some expectations on the upper tail as shown with the outliers on the boxplot below.  

In [ ]:
# Target Variable Analysis
sns.histplot(data=df, x='target', stat='frequency')
plt.title("Transaction Value Distribution")
plt.xlabel("Value ($)")
plt.show()

**Note:** As noted in the introduction, we have an anonymized dataset with all predictors standardized. Thus, we would have minimal insights in exploring them individually. 

# 3. Data Cleaning

In [ ]:
# Missing Values
df.isna().sum().sum()

**Takeaway:** With the data already formatted, we would indeed expect no null values.

In [ ]:
## Outliers
sns.boxplot(data=df, x='target')
plt.title("Transaction Value Boxplot")
plt.xlabel("Value ($)")
plt.show()

**Takeaway:** From the box plot above, We have already determined some outliers. However, we will not exclude them but use other techniques instead to ensure robust results from the machine learning predictions.

In [ ]:
# Duplicate Observations
df.duplicated().value_counts()

**Takeaway:**  With no duplicate values, we may proceed without making any further changes.

# 4. Correlation Analysis

**Takeaway:** Given that we have close to 5000 columns, it would be quite computationally intensive to display the entire correlation matrix and visualize it for user-friendly analysis. 

# 5. Machine Learning set-up

Under this section, we will explain the procedure of two main splitting approach to estimate our models' performance. 

In [ ]:
# Splitting features & target variable
X = df.drop(['target'], axis=1).values
y = df['target'].values

**Definition:** As we may imagine, variables come in different measurements like height in inches, weight in pounds, or dollar amount. To ensure a smooth run of our models, we will proceed with feature scaling by representing every value in the column based on a given reference. In our instance, we will use **StandardScaler** by subtracting from the mean and dividing by the standard deviation.

In [ ]:
# Feature Scaling
# scaler = preprocessing.StandardScaler()
# X = scaler.fit_transform(X)
# X

**Definition:** Often denoted as the most popular by its simplicity, the train-test split is a sampling technique dividing the dataset between training and testing sets. In doing so, the goal would be to have enough (but not too much) in our training set used for the machine learning model to predict the observations in the testing set as accurately as possible. Most would opt for a 70/30 training-testing split, respectively, others 80/20, 60/40, or whichever else works best for the case scenario. Further information [here](https://machinelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/). 

In [ ]:
## Training Testing Split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)


# 7. Machine Learning - Neural Networks

**Definition:** MLPRegressor

In [ ]:
mlpr = MLPRegressor(hidden_layer_sizes= (5,5,5,5),
                           max_iter=500, alpha=0.05, solver='sgd',
                           learning_rate='adaptive', activation='tanh')
mlpr.fit(X_train, y_train)
pred = mlpr.predict(X_train)
nonneg_pred = np.clip(pred, df['target'].min(), df['target'].max())
rmse = skm.mean_squared_error(y_train, nonneg_pred, squared=False) 
print('MLPRegressor - RMSE: {:.5f}' .format(rmse))

# 8. Final Submission

In [ ]:
# Loading the dataset
test_df = pd.read_csv("/kaggle/input/santander-value-prediction-challenge/test.csv")
ids = test_df['ID']
test_df.drop(['ID'], axis=1, inplace=True)

In [ ]:
# Defining our orignal model
final_model = MLPRegressor(hidden_layer_sizes= (5,5,5,5),
                           max_iter=500, alpha=0.05, solver='sgd',
                           learning_rate='adaptive', activation='tanh')
final_model.fit(X, y)
pred = final_model.predict(test_df)
nonneg_pred = np.clip(pred, df['target'].min(), df['target'].max())
sub = pd.DataFrame({'ID': ids, 'target': nonneg_pred})
sub.to_csv('submission.csv', index=False)